In [75]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
from copy import deepcopy

In [36]:
folder = Path('eoles') / Path('inputs') / Path('xps')

date = datetime.now()
date = date.strftime("%Y%m%d")  # formatting
folder = folder / Path(str(date))

# add one day to date object
# date = date + timedelta(days=1)
# check if folder is a folder with library pathlib
folder.mkdir(parents=True, exist_ok=True)

base = {
    "no_subsidies": True,
    "subsidies_specified": False,
    "calibration": True,
    "max_iter": 100,
    "health": True,
    "discount_rate": 0.032,
    "weather_year": 2006,
    "carbon_constraint": True,
    "fix_sub_heater": False,
    "fix_sub_insulation": False,
    "list_year": [2020, 2025, 2030, 2035, 2040, 2045],
    "list_trajectory_scc": [0, 250, 350, 500, 650, 775],
    "eoles": {
        "biomass_potential_scenario": "S3",
        "aggregated_potential": True,
        "maximum_capacity_scenario": "N1"
    },
    "subsidy": {
            "proportional_uniform": None,
            "heater": {
                "policy": "subsidy_ad_valorem",
                "proportional": None,
                "cap": None
            },
            "insulation": {
                "policy": "subsidy_ad_valorem",
                "rational_behavior": False,
                "target": None,
                "proportional": None,
                "cap": None
            }
    },
    "scenario_cost_eoles": {
        "fix_capacities": {
            "uiom": 0,
            "CTES": 0
        }
    }
}

folder_base = folder / Path('base.json')
with open(folder_base, "w") as outfile:
    outfile.write(json.dumps(base, indent=4))

name_mapping = {
    'biomass_potential_scenario': 'biogas',
    'maximum_capacity_scenario': 'capacity',
    'demand_scenario': 'demand',
    'hourly_profile': 'profile',
    'weather': 'weather',
    'costs_supply': 'costs'
}

# scenario_mapping = {
#     'S3': 'S3',
#     'S2': 'biogas-',
#     'N1nuc': 'nuc-',
#     'N1ren2': 'ren-',
#     'Opt': 'ren+',
#     'Reindustrialisation': 'Reindustrialisation',
#     'Sobriete': 'Sobriete',
#     'hourly_profile_flat': 'flat'
# }

dict_supply = {
    'reference': {'reference': {}},
    'biomass_potential_scenario': {'Biogas-': 'S2'},
    'maximum_capacity_scenario': {'Nuc-': 'N1nuc',
                                  'Ren-': 'N1ren2',
                                  'Ren+': 'Opt'},
    'demand_scenario': {'Reindustrialisation': 'Reindustrialisation',
                        'Sobriete': 'Sobriete'},
    'method_health_cost': {'HCHeating': 'heating_intensity'},
    'costs_supply': {
        'Hydrogen+' : {
                        'storage_capex': {'hydrogen': 0}
                    },
        'costsREN+' : {
                        'capex': 'overnight_capex_evolution_50p'
                    }
    },
    'weather' : {'2012': {'load_factors': 'vre_profiles_2012',
                  'lake_inflows': 'lake_2012',
                  'nb_years': 1,
                  'input_years': [2012]}
                 }
}

dict_demand = {
    'technical': {
        'Learning+': {
              "technical_progress": {
                "heater": {
                  "activated": True,
                  "start": 2019,
                  "end": 2035,
                  "value_end": -0.5
                }
              }
            },
        # 'VariableCostsHP':
        #     {"variable_size_heater": True},
        'ProfileFlat': {
            "hourly_profile": "project/input/technical/hourly_profile_flat.csv"
        },
    },
    'switch_heater': {
        'Elasticity-': {
          "scale": {
            "option": "price_elasticity",
            "target": -0.5
          }},
        # 'DHS2': {
        #   "district_heating": "project/input/energy/district_heating_s2.csv"
        # }
    }
}

dict_policy_demand = {
    'Ban': {
            "file": "project/input/policies/policies_2024.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
        }},
    # 'BanRef': {
    #         "file": "project/input/policies/current/policies_ref.json",
    #         "restriction_gas": {
    #             "start": 2025,
    #             "end": 2051,
    #             "value": "Natural gas",
    #             "policy": "restriction_energy"
    #     }},
    # 'BanStandalone': {
    #     "file": "project/input/policies/current/policies_ambitious.json",
    #     "restriction_indiv_gas": {
    #       "start": 2025,
    #       "end": 2051,
    #       "value": ["Natural gas-Standard boiler", "Natural gas-Performance boiler"],
    #       "policy": "restriction_heater",
    #       "target": "Single-family"
    #     }
    # },
    'BanNoPolicy': {
            "file": "project/input/policies/policies_calibration.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
        }},
    # 'CarbonTax': {
    #   "file": "project/input/policies/current/policies_calibration.json",
    #   "carbon_tax": {
    #     "start": 2017,
    #     "end": 2051,
    #     "tax": "project/input/policies/carbon_value.csv",
    #     "emission": "project/input/policies/current/carbon_tax_emission.csv",
    #     "recycling": True,
    #     "recycling_ini": 2000000000
    #   }
    # },
    'Package2024':
        {
            "file": "project/input/policies/policies_2024.json"
        },
    # 'Package2024Inertia':
    #     {
    #         "file": "project/input/policies/policies_2024.json",
    #         "inertia_heater": {
    #           "start": 2020,
    #           "end": 2051,
    #           "policy": "regulation",
    #           "gest": "heater"
    #         },
    #     },
    'NoPolicy':
        {
            "file": "project/input/policies/policies_calibration.json"
        }
}

In [76]:
deepcopy('S2')

'S2'

In [77]:
scenarios = {
    'biogas': ['reference', 'Biogas-'],
    'demand': ['reference', 'Sufficiency', 'Reindustrialisation'],
    'policy': ['Ban'],
    'learning': ['reference', 'Learning+'],
    'profile': ['reference', 'ProfileFlat']
}

map_values = {
    'Biogas-': 'S2',
    'Sufficiency': 'Sobriete',
    'Reindustrialisation': 'Reindustrialisation',
    'Ban': {
        "file": "project/input/policies/policies_2024.json",
        "restriction_gas": {
            "start": 2025,
            "end": 2051,
            "value": "Natural gas",
            "policy": "restriction_energy"
    }},
    'Learning+': {
          "technical_progress": {
            "heater": {
              "activated": True,
              "start": 2019,
              "end": 2035,
              "value_end": -0.5
            }
          }
        },
    'ProfileFlat': {
            "hourly_profile": "project/input/technical/hourly_profile_flat.csv"
        }
}

mapping_scenarios_to_configs = {
    'biogas': ['supply', 'biomass_potential_scenario'],
    'policy': ['policies'],
    'demand': ['supply', 'demand_scenario'],
    'technical': ['demand'],
    'learning': ['demand', 'technical'],
    'profile': ['demand', 'technical']
}



In [78]:
keys, values = zip(*scenarios.items())

In [79]:
import itertools
permutations_names = [dict(zip(keys, v)) for v in itertools.product(*values)]
permutations_names = {'S{}'.format(n): v for n, v in enumerate(permutations_names)}

In [80]:
permutations_names

{'S0': {'biogas': 'reference',
  'demand': 'reference',
  'policy': 'Ban',
  'learning': 'reference',
  'profile': 'reference'},
 'S1': {'biogas': 'reference',
  'demand': 'reference',
  'policy': 'Ban',
  'learning': 'reference',
  'profile': 'ProfileFlat'},
 'S2': {'biogas': 'reference',
  'demand': 'reference',
  'policy': 'Ban',
  'learning': 'Learning+',
  'profile': 'reference'},
 'S3': {'biogas': 'reference',
  'demand': 'reference',
  'policy': 'Ban',
  'learning': 'Learning+',
  'profile': 'ProfileFlat'},
 'S4': {'biogas': 'reference',
  'demand': 'Sufficiency',
  'policy': 'Ban',
  'learning': 'reference',
  'profile': 'reference'},
 'S5': {'biogas': 'reference',
  'demand': 'Sufficiency',
  'policy': 'Ban',
  'learning': 'reference',
  'profile': 'ProfileFlat'},
 'S6': {'biogas': 'reference',
  'demand': 'Sufficiency',
  'policy': 'Ban',
  'learning': 'Learning+',
  'profile': 'reference'},
 'S7': {'biogas': 'reference',
  'demand': 'Sufficiency',
  'policy': 'Ban',
  'learn

In [84]:
config_additional = {
    "name_config": "reference",
    "policies": {
        "file": "project/input/policies/current/policies_ambitious.json"
    },
    "greenfield": False,
    "prices_constant": False,
    "price_feedback": True,
    "biomass_potential_scenario": "S3",
    "aggregated_potential": True,
    "maximum_capacity_scenario": "N1",
    "lifetime_insulation": 5,
    "subsidies_heater": None,
    "subsidies_insulation": None,
    "optim_eoles": True,
    "carbon_budget": "carbon_budget_tight_convex",
    "district_heating_potential": None,
    "dict_configs": None,
    "cap_MWh": 1000,
    "cap_tCO2": 2000,
    "demand_scenario": "Reference",
    "method_health_cost": "epc"
}

for key, values in permutations_names.items():  #key: 'S0', values= 'biogas'
    new_config = config_additional.copy()
    for i,v in values.items():  # i: 'biogas', v: 'Biogas-'
        if v == 'reference':  # no modification to the configuration
            pass
        else:
            if mapping_scenarios_to_configs[i][0] == 'supply':
                new_config[mapping_scenarios_to_configs[i][1]] = deepcopy(map_values[v])
                # print(new_config)
            elif mapping_scenarios_to_configs[i][0] == 'demand':
                if mapping_scenarios_to_configs[i][1] in new_config.keys():
                    new_config[mapping_scenarios_to_configs[i][1]].update(deepcopy(map_values[v]))
                else:
                    new_config[mapping_scenarios_to_configs[i][1]] = deepcopy(map_values[v])
            elif mapping_scenarios_to_configs[i][0] == 'policies':
                new_config['policies'] = deepcopy(map_values[v])
            else:
                raise KeyError('Key not found')
    folder_additional = folder / Path(key + '.json')
    with open(folder_additional, "w") as outfile:
        outfile.write(json.dumps(new_config, indent=4))

pd.DataFrame.from_dict(permutations_names, orient='index').to_csv(folder / Path('scenarios.csv'))

In [3]:
config_additional = {
    "name_config": "reference",
    "policies": {
        "file": "project/input/policies/current/policies_ambitious.json"
    },
    "greenfield": False,
    "prices_constant": False,
    "price_feedback": True,
    "biomass_potential_scenario": "S3",
    "aggregated_potential": True,
    "maximum_capacity_scenario": "N1",
    "lifetime_insulation": 5,
    "subsidies_heater": None,
    "subsidies_insulation": None,
    "optim_eoles": True,
    "carbon_budget": "carbon_budget_tight_convex",
    "district_heating_potential": None,
    "dict_configs": None,
    "cap_MWh": 1000,
    "cap_tCO2": 2000,
    "demand_scenario": "Reference",
    "method_health_cost": "epc"
}

list_configs = []
for supply, scenario in dict_supply.items():
    for name, s in scenario.items():
        print(name)
        for demand, policy in dict_policy_demand.items():
            new_config = config_additional.copy()
            new_config['policies'] = policy
            if name == 'reference':
                pass
            else:
                new_config[supply] = s
            name_config = '_'.join([name, f'policy{demand}'])
            print(name_config)
            # name_config = '_'.join([new_name if name_mapping[supply] in x else x for x in new_config['name_config'].split('_')])
            # print(name_config)
            # name_config = name_config + f'_policy{demand}'
            new_config['name_config'] = name_config
            list_configs.append(new_config)
            folder_additional = folder / Path(name_config + '.json')
            with open(folder_additional, "w") as outfile:
                outfile.write(json.dumps(new_config, indent=4))

for supply, scenario in dict_demand.items():
    for s, name in zip(scenario.values(), [n for n in scenario.keys()]):
        print(name)
        for demand, policy in dict_policy_demand.items():
            new_config = config_additional.copy()
            new_config[supply] = s
            new_config['policies'] = policy
            if scenario == 'reference':
                name_config = 'reference'
            else:
                name_config = '_'.join([name, f'policy{demand}'])
            print(name_config)
            new_config['name_config'] = name_config
            list_configs.append(new_config)
            folder_additional = folder / Path(name_config + '.json')
            with open(folder_additional, "w") as outfile:
                outfile.write(json.dumps(new_config, indent=4))


# list_configs = []
# for supply, scenario in dict_supply.items():
#     for s in scenario:
#         for demand, policy in dict_demand.items():
#             print(supply, s)
#             new_config = config_additional.copy()
#             new_config[supply] = s
#             new_config['policies'] = policy
#             new_name = name_mapping[supply] + scenario_mapping[s]
#             print(new_name)
#             name_config = '_'.join([new_name if name_mapping[supply] in x else x for x in new_config['name_config'].split('_')])
#             print(name_config)
#             name_config = name_config + f'_policy{demand}'
#             new_config['name_config'] = name_config
#             list_configs.append(new_config)
#             folder_additional = folder / Path(name_config + '.json')
#             with open(folder_additional, "w") as outfile:
#                 outfile.write(json.dumps(new_config, indent=4))

reference
reference_policyBan
reference_policyBanNoPolicy
reference_policyPackage2024
reference_policyNoPolicy
Biogas-
Biogas-_policyBan
Biogas-_policyBanNoPolicy
Biogas-_policyPackage2024
Biogas-_policyNoPolicy
Nuc-
Nuc-_policyBan
Nuc-_policyBanNoPolicy
Nuc-_policyPackage2024
Nuc-_policyNoPolicy
Ren-
Ren-_policyBan
Ren-_policyBanNoPolicy
Ren-_policyPackage2024
Ren-_policyNoPolicy
Ren+
Ren+_policyBan
Ren+_policyBanNoPolicy
Ren+_policyPackage2024
Ren+_policyNoPolicy
Reindustrialisation
Reindustrialisation_policyBan
Reindustrialisation_policyBanNoPolicy
Reindustrialisation_policyPackage2024
Reindustrialisation_policyNoPolicy
Sobriete
Sobriete_policyBan
Sobriete_policyBanNoPolicy
Sobriete_policyPackage2024
Sobriete_policyNoPolicy
HCHeating
HCHeating_policyBan
HCHeating_policyBanNoPolicy
HCHeating_policyPackage2024
HCHeating_policyNoPolicy
Hydrogen+
Hydrogen+_policyBan
Hydrogen+_policyBanNoPolicy
Hydrogen+_policyPackage2024
Hydrogen+_policyNoPolicy
costsREN+
costsREN+_policyBan
costsREN+_p